In [2]:
# importing all the modules
from cvzone.HandTrackingModule import HandDetector
import cv2
import tensorflow as tf
import numpy as np
import pickle
import time

In [3]:
# loading the saved model Name(Sign_Model.h5') 
model = tf.keras.models.load_model(r'Sign_Model.h5')

In [4]:
labels = ['0','1','2','3','4','5','6','7','8','9','A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M','N','O', 'P', 'Q','R', 'S', 'T','U', 'V', 'W', 'X', 'Y', 'Z']

In [5]:
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8, maxHands=2)
word = ""
words = []
timer = time.time()

while True:

# Get image frame
    success, img = cap.read()
    #flip the image
    img = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_BGR2RGB)
    #storing the results
    #results = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # Find the hand and its landmarks
    #hands, img = detector.findHands(img) # with draw
    hands = detector.findHands(img, draw=False) # without draw
    
    
    
    if hands:
        # Hand 1
        hand1 = hands[0]
        
        bbox1 = hand1["bbox"]  # Bounding box info x,y,w,h
        centerPoint1 = hand1['center']  # center of the hand cx,cy
        
        

        
        if len(hands)>=2:
            hand2 = hands[1]
            bbox2 = hand2["bbox"]
            centerPoint2 = hand2['center']

            bbox1_height = hands[0]["bbox"][3]  # Get the height of bbox1
            bbox2_height = hands[1]["bbox"][3]  # Get the height of bbox2
           
            x1 = min(bbox1[0], bbox2[0])
            y1 = min(bbox1[1], bbox2[1])
            x2 = max(bbox1[0] + bbox1[2], bbox2[0] + bbox2[2])
            y2 = max(bbox1[1] + bbox1[3], bbox2[1] + bbox2[3])
           
           
            offset_x, offset_y = 30, 30
            x1 -= offset_x
            y1 -= offset_y
            x2 += offset_x
            y2 += offset_y
           
            combined_bbox = [x1, y1, x2-x1, y2-y1]
           
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
           
            imgCrop = img[y1:y2,x1:x2]
            image1 = cv2.resize(imgCrop, (64, 64))
            image2 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

            image3 = cv2.Canny(image2, 60, 120)
            image4 = image3.astype(np.float32)/255.0
            image = np.expand_dims(image4, axis=0)
            
            #index = pickle.load(open("index.pickle", "rb"))
            
            '''output = model.predict(image)

            predicted_class = output.argmax()
            #labels = list(index.keys())
            predicted_letter = labels[predicted_class] 
            
            
            if time.time() - timer >= 2:
                word += predicted_letter
                timer = time.time()
                
            if predicted_letter == "":
                words.append(word)
                word = ""'''
            
            #cv2.putText(imgCrop, word, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

            #cv2.putText(imgCrop, 'Predicted: ' + predicted_letter, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        
        if len(hands)==1:
            hand1 = hands[0]
            x, y, w, h = hand1["bbox"]
            offset = 30
            cv2.rectangle(img, (x-offset,y-offset), ((w+x+offset), (h+y+offset)), (255, 0 , 0), 2)
            imgCrop = img[y-offset:y+h+offset, x-offset:x+w+offset]

            image1 = cv2.resize(imgCrop, (64, 64))
            image2 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

            image3 = cv2.Canny(image2, 60, 120)
            cv2.imshow("Canny", image3)
            image4 = image3.astype(np.float32)/255.0
            image = np.expand_dims(image4, axis=0)
            
            #index = pickle.load(open("index.pickle", "rb")
                
            
    predicted_letter = "" 
    
    output = model.predict(image)

    predicted_class = output.argmax()
            
    predicted_letter = labels[predicted_class] 
    
    cv2.putText(imgCrop, 'Predicted: ' + predicted_letter, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    #cv2.putText(img, 'Predicted: ' + predicted_letter, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)
            
    #cv2.putText(imgCrop, word, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
    
    if len(hands)==0:
        predicted_letter = ""
                  
    if len(predicted_letter) != 0 and time.time() - timer >= 5:
        word += predicted_letter
        timer = time.time()
        cv2.putText(img, word, (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        
    if len(hands) == 0 and len(word)>0:
        words.append(word)
        word = ""
        #cv2.putText(img, words, (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (24, 255, 5), 2)
        
                
    '''if len(predicted_letter) == 0:
        words.append(word)
        word = ""'''
    
    
    cv2.imshow("Image", img)
    
    '''for word in words:
        cv2.putText(img, 'Recognized words:' + word, (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)'''
    

    if cv2.waitKey(1) & 0xFF == ord('q'):  # exit the loop when 'q' is pressed
        break



cap.release()
cv2.destroyAllWindows()

print("Recognized words: ", words)




1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 21ms/step
Recognized words:  ['5LLS', 'JE']


In [6]:
cap.release()
cv2.destroyAllWindows()
